In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import t

In [2]:
# Read the data and seperate by ;
data = pd.read_csv('gdp_data.csv', sep=';')

data.drop(data.columns[0], axis=1, inplace=True)

display(data.head())

,income,capital,labor
0,114043,8310,182113
1,120410,8529,193749
2,129187,8738,205192
3,134705,8952,215130
4,139960,9171,225021


In [7]:
data['ln_capital'] = np.log(data['capital'])
data['ln_labor'] = np.log(data['labor'])

display(data.head())

,income,capital,labor,ln_capital,ln_labor
0,114043,8310,182113,9.025215,12.112383
1,120410,8529,193749,9.051227,12.174319
2,129187,8738,205192,9.075437,12.231701
3,134705,8952,215130,9.099632,12.278998
4,139960,9171,225021,9.123802,12.323949


In [22]:
# Add constant for the intercept term
Y = data['income']
X = data[['ln_capital', 'ln_labor']]
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(Y, X)

results = model.fit()

# Print model summary
print(results.summary())

# Print coefficients
print('Coefficients:', results.params)

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     971.4
Date:                Mon, 08 Apr 2024   Prob (F-statistic):           2.99e-18
Time:                        22:09:22   Log-Likelihood:                -206.83
No. Observations:                  20   AIC:                             419.7
Df Residuals:                      17   BIC:                             422.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.518e+06   1.76e+05    -14.271      0.0

In [21]:
plot = False

if plot:
    fitted_values = results.fittedvalues 
    residuals = results.resid

    # plotting the residuals and fitted values
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.scatter(fitted_values, residuals)
    plt.xlabel('Fitted values')
    plt.ylabel('Residuals')
    plt.title('Residuals vs Fitted')

    # plotting the residuals
    plt.subplot(1, 2, 2)
    plt.hist(residuals, bins=50)
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.title('Histogram of Residuals')
    plt.tight_layout()
    plt.show()

Overall, based on the high R-squared value, low p-value of the F-statistic, and significant coefficient of ln_labor, we can conclude that the model provides an excellent fit to the data in explaining the variation in income. However, the non-significant coefficient for ln_capital suggests that this variable might not be a useful predictor of income in this model. Additionally, the low Durbin-Watson statistic indicates potential issues with autocorrelation in the residuals, which might need further investigation or model refinement.

In [24]:
yhat = results.predict(X)
uhat = Y - yhat

# preparation to MonteCarlo
MC = 10000; BS = len(data)
df = len(X.columns)

# this is the correction suggested by Davidson Mc Kinnon(94)
u_hatbs = np.sqrt((BS/(BS-df))) * uhat

# Initialize arrays for storing results
y_vec = np.full((MC, BS),np.nan)
betas0 = np.full(MC,np.nan)
betas1 = np.full(MC,np.nan)
betas2 = np.full(MC,np.nan)

sdu = np.sqrt(np.sum((uhat ** 2) / (BS - 3)))

0     17484.273135
1     10237.893912
2      6403.681945
3      1578.409051
4     -2982.828737
5     -3606.521516
6     -7262.945505
7    -10088.844531
8     -8494.202856
9     -2488.689500
10    -3781.901129
11    -4460.790602
12    -6879.041859
13    -3478.376345
14    -3506.285489
15     -596.309289
16    -4246.642132
17     2379.161724
18    10704.676617
19    13085.283105
dtype: float64

In [ ]:
for mc in range(MC):
    uboot = np.random.choice(u_hatbs, BS, replace=True) 
    u_normal = np.random.normal(size=BS) * sdu
    u_t = t.rvs(df, size=BS) * sdu
    # t with 3 degrees of freedom
    
    # MC
    y_tmp = results.params[0] + results.params[1]*data['capital'] + results.params[2]*data['labor'] + u_t
    y_vec[mc, :] = y_tmp

    # Fit the model
    X_tmp = sm.add_constant(np.column_stack((data['capital'], data['labor'])))
    fit_tmp = sm.OLS(y_tmp, X_tmp).fit()
    
    betas0[mc] = fit_tmp.params[0] 
    betas1[mc] = fit_tmp.params[1] 
    betas2[mc] = fit_tmp.params[2]